### 实现影像的裁剪、拼接

In [20]:
from osgeo import gdal
from osgeo import osr
import numpy as np
import matplotlib.pyplot as plt


In [21]:
dset = gdal.Open('data/Section_5/rsimg_chenggong_wgs84.tif')
geo_trans = dset.GetGeoTransform()   ### 地理转换参数
print('geotrans:', geo_trans)
x_min, y_max = geo_trans[0], geo_trans[3]   ### 左上角坐标
x_max = geo_trans[0]+geo_trans[1]*dset.RasterXSize
y_min = geo_trans[3]+geo_trans[5]*dset.RasterYSize
print('extent:', x_min, x_max, y_min, y_max)


geotrans: (102.703814432, 0.00018789276986649867, 0.0, 24.993242602, 0.0, -0.00018789276977491845)
extent: 102.703814432 102.999369759 24.701069345 24.993242602


In [23]:
### 设定裁剪范围
extent_subs_1 = [102.703814432, 102.999369759, 24.701069345, 24.86]  #



In [24]:
#### 计算影像尺寸
x_size_subs1 = (102.999369759-102.703814432)/geo_trans[1]
y_size_subs1 = (24.701069345-24.86)/geo_trans[5]
print(x_size_subs1, y_size_subs1)
x_size_subs1, y_size_subs1 = 1573, 846
x_res = (102.999369759-102.703814432)/1573
y_res = (24.701069345-24.86)/846
print(x_res, y_res)


1573.0 845.8582796474075
0.00018789276986649867 -0.00018786129432623986


In [25]:
geotrans_subs1 = [102.703814432, 0.00018789276986649867, 0, 24.86, 0, -0.00018786129432623986]


In [46]:
### 获取裁剪影像数组
img_array = dset.ReadAsArray()
print(img_array.shape)  ## （波段数，行，列) -> (波段数,y,x)
### 计算裁剪影像起点
## subs1_x_start, subs_y_start -> 102.703814432, 24.86
## img_x_start, img_y_start -> 102.703814432, 24.993242602
col_dif_img_sbus1_start = (102.703814432 - 102.703814432)/geotrans_subs1[1]
row_dif_img_sbus1_start = (24.86 - 24.993242602)/geotrans_subs1[5]
print(col_dif_img_sbus1_start, row_dif_img_sbus1_start)

(6, 1555, 1573)
0.0 709.2605343632425
(6, 846, 1573)


In [50]:
### 行列间隔数不是整数，通过调整裁剪影像起点对其整数化。
col_dif_img_sbus1_start, row_dif_img_sbus1_start = 0, 709
x_start_subs1_new = 102.703814432 + col_dif_img_sbus1_start * geotrans_subs1[1]
y_start_subs1_new = 24.993242602 + row_dif_img_sbus1_start * geotrans_subs1[5]
print(x_start_subs1_new, y_start_subs1_new)


102.703814432 24.860048944322696


In [ ]:
### 获取裁剪影像数组
### 设定x,y方向上裁剪影像和原始影像左上角像元个数差异为0，709，获得裁剪影像数组
img_array_subs1 = img_array[:, 709:709+846, 0:0+1573]
print(img_array_subs1.shape)


In [28]:
### 更新geotrans
geotrans_subs1 = [102.703814432, 0.00018789276986649867, 0, 24.860048944322696, 0, -0.00018786129432623986]


In [39]:
driver = gdal.GetDriverByName('GTiff')
dset_subs1 = driver.Create('data/Section_5/rsimg_chenggong_wgs84_subs1.tif', xsize = 1573, \
                                        ysize = 846, bands=dset.RasterCount, eType=gdal.GDT_Int16)
dset_subs1.SetGeoTransform(geotrans_subs1)

dset_subs1.SetProjection(dset.GetProjection())
dset_subs1.WriteArray(img_array_subs1)      ### 将np.array()数组写入数据
dset_subs1 = None


## 课堂练习：基于gdal库练习给定范围的影像裁剪
给定范围为：extent_subs_2 = [102.703814432, 102.999369759, 24.84, 24.993242602]

In [45]:
dset = gdal.Open('data/Section_5/rsimg_chenggong_wgs84.tif')
geo_trans = dset.GetGeoTransform()   ### 地理转换参数
print(geo_trans)

(102.703814432, 0.00018789276986649867, 0.0, 24.993242602, 0.0, -0.00018789276977491845)


In [63]:
extent_subs_2 = [102.71, 102.99, 24.84, 24.99]


In [64]:
#### 计算影像尺寸
x_size_subs2 = (102.99-102.71)/geo_trans[1]
y_size_subs2 = (24.84-24.99)/geo_trans[5]
print(x_size_subs2, y_size_subs2)


1490.2116787087996 798.3276854116708


In [65]:
### 裁剪影像尺寸整数化
x_size_subs2, y_size_subs2 = 1490, 798
x_res = (102.99-102.71)/x_size_subs2
y_res = (24.84-24.99)/y_size_subs2
print(x_res, y_res)



0.0001879194630872491 -0.0001879699248120283


In [66]:
### 更新geotrans
geotrans_subs2 = [102.71, 0.0001879194630872491, 0, 24.99, 0, -0.0001879699248120283]


In [67]:
### 获取裁剪影像数组, 首先计算裁剪影像起点
col_dif_img_subs2_start = (102.71-102.703814432)/geo_trans[1]
row_dif_img_subs2_start = (24.99 - 24.993242602)/geo_trans[5]
print(col_dif_img_subs2_start, row_dif_img_subs2_start)


32.92073454655664 17.25772632914325


In [68]:
### 行列差异整数化, 并更新裁剪影像起点
col_dif_img_subs2_start, row_dif_img_subs2_start = 32, 17
x_start_subs2_new = 102.703814432 + col_dif_img_subs2_start*geo_trans[1]
y_start_subs2_new = 24.993242602 + row_dif_img_subs2_start*geo_trans[5]
print(x_start_subs2_new, y_start_subs2_new)


102.70982700063573 24.990048424913827


In [70]:
img_array_subs2 = img_array[:,17:17+798, 32:32+1490]
print(img_array_subs2.shape)


(6, 798, 1490)


In [69]:
### 更新geotrans
geotrans_subs2 = [102.70982700063573, 0.0001879194630872491, 0, 24.990048424913827, 0, -0.0001879699248120283]


In [71]:
driver = gdal.GetDriverByName('GTiff')
dset_subs2 = driver.Create('data/Section_5/rsimg_chenggong_wgs84_subs2.tif', xsize = x_size_subs2, \
                                        ysize = y_size_subs2, bands=dset.RasterCount, eType=gdal.GDT_Int16)
dset_subs2.SetGeoTransform(geotrans_subs2)
dset_subs2.SetProjection(dset.GetProjection())
dset_subs2.WriteArray(img_array_subs2)      ### 将np.array()数组写入数据
dset_subs2 = None


### 影像拼接

In [74]:
dset_subs1 = gdal.Open('data/Section_5/rsimg_chenggong_wgs84_subs1.tif')
dset_subs2 = gdal.Open('data/Section_5/rsimg_chenggong_wgs84_subs2.tif')
geo_trans_subs1 = dset_subs1.GetGeoTransform()   ### 地理转换参数
geo_trans_subs2 = dset_subs2.GetGeoTransform()   ### 地理转换参数
print('subs1:', geo_trans_subs1)
print('subs2:', geo_trans_subs2)


subs1: (102.703814432, 0.00018789276986649867, 0.0, 24.860048944322696, 0.0, -0.00018786129432623986)
subs2: (102.70982700063573, 0.0001879194630872491, 0.0, 24.990048424913827, 0.0, -0.0001879699248120283)


In [78]:
x_right_subs1 = 102.703814432 + dset_subs1.RasterXSize * geo_trans_subs1[1]
y_bottom_subs1 = 24.860048944322696 + dset_subs1.RasterYSize * geo_trans_subs1[5]
print('extent_subs1:', 102.703814432, x_right_subs1, y_bottom_subs1, 24.860048944322696 )

x_right_subs2 = 102.70982700063573 + dset_subs2.RasterXSize * geo_trans_subs2[1]
y_bottom_subs2 = 24.990048424913827 + dset_subs2.RasterYSize * geo_trans_subs2[5]
print('extent_subs2:', 102.70982700063573, x_right_subs2, y_bottom_subs2, 24.990048424913827)


extent_subs1: 102.703814432 102.999369759 24.701118289322697 24.860048944322696
extent_subs2: 102.70982700063573 102.98982700063573 24.840048424913828 24.990048424913827


In [80]:
extent_mosaic = [102.703814432, 102.999369759, 24.701118289322697, 24.990048424913827]


In [84]:
x_res, y_res = 0.00018789276986649867, -0.00018786129432623986
x_size_mosaic = (extent_mosaic[1] - extent_mosaic[0])/x_res
y_size_mosaic = (extent_mosaic[2]-extent_mosaic[3])/y_res
print(x_size_mosaic, y_size_mosaic)


1573.0 1537.9971517143601


In [88]:
### 尺寸整数化
x_size_mosaic, y_size_mosaic = 1573, 1538
x_res_new = (extent_mosaic[1] - extent_mosaic[0])/x_size_mosaic
y_res_new = (extent_mosaic[2]-extent_mosaic[3])/y_size_mosaic
print(x_res_new, y_res_new)


0.00018789276986649867 -0.00018786094641815997


In [89]:
geotrans_mosaic = [102.703814432,0.00018789276986649867,0,24.990048424913827,0,-0.00018786094641815997]


In [90]:
img_array_mosaic = np.zeros(shape=(dset_subs1.RasterCount, y_size_mosaic, x_size_mosaic))
img_array_mosaic.shape


(6, 1538, 1573)

In [94]:
### 待拼接影像起点在拼接影像上位置
x_start_subs1, y_start_subs1 = 102.703814432, 24.860048944322696
x_start_subs2, y_start_subs2 = 102.70982700063573, 24.990048424913827
x_start_mosaic, y_start_mosaic = 102.703814432, 24.990048424913827
### subs1
row_start_subs1 = (y_start_subs1 - y_start_mosaic)/y_res
col_start_subs1 = (x_start_subs1 - x_start_mosaic)/x_res
print(row_start_subs1, col_start_subs1)
img_array_mosaic[:, 692:692+dset_subs1.RasterYSize, 0:0+dset_subs1.RasterXSize] = dset_subs1.ReadAsArray()
### subs2
row_start_subs2 = (y_start_subs2 - y_start_mosaic)/y_res
col_start_subs2 = (x_start_subs2 - x_start_mosaic)/x_res
print(row_start_subs2, col_start_subs2)
img_array_mosaic[:, 0:0+dset_subs2.RasterYSize, 32:32+dset_subs2.RasterXSize] = dset_subs2.ReadAsArray()



691.9971517143603 0.0
-0.0 31.999999999985864


In [96]:
driver = gdal.GetDriverByName('GTiff')
dset_mosaic = driver.Create('data/Section_5/rsimg_chenggong_wgs84_subs_mosaic.tif', xsize = x_size_mosaic, \
                                        ysize = y_size_mosaic, bands=dset_subs1.RasterCount, eType=gdal.GDT_Int16)
dset_mosaic.SetGeoTransform(geotrans_mosaic)
dset_mosaic.SetProjection(dset_subs1.GetProjection())
dset_mosaic.WriteArray(img_array_mosaic)      ### 将np.array()数组写入数据
dset_mosaic = None



### 快捷方式
影像裁剪: gdal_translate (参考：https://gdal.org/programs/gdal_translate.html)

In [98]:
### extent: str(ulx) str(uly) str(lrx) str(lry)
extent = '102.703814432 24.86 102.999369759 24.701069345'
path_in = 'data/Section_5/rsimg_chenggong_wgs84.tif'
path_out = 'data/Section_5/rsimg_chenggong_wgs84_subs1_.tif'
!gdal_translate -projwin $extent -co COMPRESS=LZW $path_in $path_out


Input file size is 1573, 1555
0...10...20...30...40...50...60...70...80...90...100 - done.


#### 影像拼接：gdal_merge (参考：https://gdal.org/programs/gdal_merge.html)

In [99]:
path_out = 'data/Section_5/rsimg_chenggong_wgs84_subs_mosaic_.tif'
path_in_1 = 'data/Section_5/rsimg_chenggong_wgs84_subs1.tif'
path_in_2 = 'data/Section_5/rsimg_chenggong_wgs84_subs2.tif'
!gdal_merge.py -init 0 -co COMPRESS=LZW -o $path_out $path_in_1 $path_in_2


0...10...20...30...40...50...60...70...80...90...100 - done.
